# Welcome to Python in CoCalc!

Run a cell via `Shift + Return`. Learn more about [CoCalc Jupyter Notebooks](https://doc.cocalc.com/jupyter.html).

In [67]:
def slice_image(im,M=1,N=1):
    tiles = [im[x:x+M,y:y+N].flatten() for x in range(0,im.shape[0],M) for y in range(0,im.shape[1],N)]
    return tiles

def read_image(fn="./images/017.png_wave.jpg"):
    import cv2
    return cv2.imread(fn,cv2.COLOR_BGR2RGB) # grayscale




def constructImageData(Ndim=11,imageFN="./data/image.jpg",nr=10):
    import numpy as np
    from sklearn.decomposition import PCA
    
    #import umap
    #reducer = umap.UMAP(n_neighbors=15,min_dist=0.1,n_components=2*len(tracks),metric="euclidean",random_state=42)
    pca = PCA(n_components=Ndim)
    #from sklearn import datasets
    import pandas as pd

    import numpy as np
    from sklearn.impute import SimpleImputer

    fn = imageFN
    #fn = "./300/image.jpg"
    #nr = 64
    im = read_image(fn)
    N = im.shape[1]//nr
    mydata = slice_image(im,M=im.shape[0],N=N)
    print(len(mydata))
    #X = mydata.iloc[:,2:]
    Xboston = mydata #mydata.data
    X = [x/np.sqrt(np.dot(x,x)) if np.dot(x,x)>0.0 else x for x in Xboston]


    from sklearn.decomposition import PCA

    nDim =Ndim
    pca = PCA(n_components=nDim)
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    stdScaler = StandardScaler() 
    
    
    X = pca.fit_transform(stdScaler.fit_transform(X))
    #y = mydata.iloc[:,1].values
    #y = mydata.target
    return X


In [59]:
def tsp(data):
    import numpy as np
    from python_tsp.distances import great_circle_distance_matrix

    sources = np.array(data)
    distance_matrix = great_circle_distance_matrix(sources)
    from python_tsp.distances import tsplib_distance_matrix
    from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing

    permutation, distance = solve_tsp_simulated_annealing(distance_matrix)
    return permutation, distance


def constructData(Ndim=12):
    from sklearn.datasets import load_boston
    Xboston, y = load_boston(return_X_y=True)
    X = [x/np.sqrt(np.dot(x,x)) for x in Xboston]


    from sklearn.decomposition import PCA

    nDim =Ndim
    pca = PCA(n_components=nDim)
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    stdScaler = StandardScaler() 
    
    
    X = pca.fit_transform(stdScaler.fit_transform(X))
    #y = mydata.iloc[:,1].values
    #y = mydata.target
    return X


In [170]:
#from sage.all import *

import numpy as np

import pandas,sys

import statsmodels.api as sm

from statsmodels.tsa.api import VAR

def parseMidi(fp,part=0):
    import os
    from music21 import converter
    #print(fp)
    score = converter.parse(fp,quantizePost=True)
    #print(list(score.elements[0].notesAndRests))
    score.makeVoices()
    #print(dir(score))
    #print([e.partAbbreviation for e in score.elements][0])
    from music21 import chord
    durs = []
    ll0 = []
    vols = []
    isPauses = []
    for p in score.elements[part].recurse().notesAndRests: #parts[0].streams()[part].notesAndRests:
        #print(p)
        if type(p)==chord.Chord:
            pitches = sorted([e.pitch.midi-21 for e in p])[0] # todo: think about chords
            vol = sorted([e.volume.velocity for e in p])[0]
            dur = float(p.duration.quarterLength)
            #print(pitches)
            
            isPause = False
        elif (p.name=="rest"):
            pitches = 89
            vol = 64
            dur = float(p.duration.quarterLength)
            
            isPause = True
        else:
            pitches = p.pitch.midi-21
            vol = p.volume.velocity
            dur = float(p.duration.quarterLength)
            
            isPause =  False
        if not dur>0 and vol>0:
            continue
        ll0.append(min(87,pitches))    
        durs.append(dur/(12*4.0))
        if vol is None or vol == 0:
            vol = 64
        vols.append(vol*1.0/127.0)
        isPauses.append(isPause)
        #print(pitches,dur,vol,isPause)
            #print(p.name,p.octave,p.duration.quarterLength)
    #print(dir(score)) 
    #print(ll0)
    #print(durs)
    return ll0,durs,vols,isPauses



def dist(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return sqrt(2*(1-gcd(a,b)^2/(a*b)))

def kernPause(a1,a2):
    return  1*(a1==a2)

def kernPitch(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return gcd(a,b)**2/(a*b)

def kernDuration(k1,k2):
    if k1==1.0 or k2==1.0:
        return 0.0
    return  log(k1)*log(k2)/sqrt(log(k1)**2*log(k2)**2)

def kernVolume(v1,v2):
    if v1==1.0 or v2==1.0:
        return 0.0
    return log(v1)*log(v2)/sqrt(log(v1)**2*log(v2)**2)

def kern(t1,t2):
    pitch1,duration1,volume1,isPause1 = t1
    pitch2,duration2,volume2,isPause2 = t2
    return 1.0/4*(kernPause(isPause1,isPause2)+kernPitch(pitch1,pitch2)+kernDuration(duration1,duration2)+kernVolume(volume1,volume2))

def getRational(k):
    alpha = 2**(1/12.0)
    x = RDF(alpha**k).n(50)
    return x.nearby_rational(max_error=0.01*x)


def ngrams(input, n):
    output = []
    for i in range(len(input)-n+1):
        output.append(input[i:i+n])
    return output

def kernNgram(ngrams1,ngrams2):
    return 1.0/len(ngrams1)*sum([ kern(ngrams1[i], ngrams2[i]) for i in range(len(ngrams1))]) 


#a#lpha = (2**(1/12.0))
#qq=[RDF(alpha**k).n(50).nearby_rational(max_error=0.01*RDF(alpha**k).n(50)) for k in range(12)]


def getBestLength(fn,Nmaxlen=20,part=1):
    pitches, durations, volumes, isPauses = parseMidi(fn,part=part)
    ll = []
    zz = list(zip(pitches,durations,volumes,isPauses))
    #zz2 = list(zip(pitches2,durations2,volumes2,isPauses2))
    #print(len((zz)))
    #M = matrix([[1]])
    Nlen = len(zz)
    s = 0
    for Nseq in range(1,Nmaxlen):
        Z = ngrams([t for t in zz[0:Nmaxlen]],Nseq)
        n = len(Z)
    #Z.extend(ngrams([t for t in zz2[0:Nmaxlen2]],Nseq))
        M = matrix([[kernNgram(t1,t2)-0.5 for t1 in Z] for t2 in Z],ring=RDF)
        #print(M)
        s=sum(sum(M))/n**2
        ll.append((s,Nseq))
    return sorted(ll,reverse=True)   
      

def measureConsonance(pitches,durations,volumes,isPauses,Nmaxlen):
    zz = list(zip(pitches,durations,volumes,isPauses))
    #zz2 = list(zip(pitches2,durations2,volumes2,isPauses2))
    #print(len((zz)))
    #M = matrix([[1]])
    Nlen = len(zz)
    s = 0
    for Nseq in range(1,Nmaxlen):
        Z = ngrams([t for t in zz[0:Nmaxlen]],Nseq)
    #Z.extend(ngrams([t for t in zz2[0:Nmaxlen2]],Nseq))
        M = matrix([[kernNgram(t1,t2) for t1 in Z] for t2 in Z],ring=RDF)
        #print(M)
        s+=sum(sum(M))/(Nmaxlen-Nseq+1)**2.0
    return s/(Nmaxlen*1.0)    
    
def getGramMatrix(pitches,durations,volumes,isPauses,Nmaxlen,Nseq):
    zz = list(zip(pitches,durations,volumes,isPauses))
    #zz2 = list(zip(pitches2,durations2,volumes2,isPauses2))
    print(len((zz)))
    #M = matrix([[1]])
    Nlen = len(zz)
    Z = ngrams([t for t in zz[0:Nmaxlen]],Nseq)
    #Z.extend(ngrams([t for t in zz2[0:Nmaxlen2]],Nseq))
    M = matrix([[kernNgram(t1,t2) for t1 in Z] for t2 in Z],ring=RDF)
    if not M.is_positive_definite():
        M+= matrix.identity(len(Z))*0.1
    return M
    
def similarity(fn1,fn2,part=0,Nmaxlen=50):
    pitches,durations,volumes,isPauses = parseMidi(fn1,part=part)
    pitches2,durations2,volumes2,isPauses2 = parseMidi(fn2,part=part)
    
    zz = list(zip(pitches,durations,volumes,isPauses))
    zz2 = list(zip(pitches2,durations2,volumes2,isPauses2))
    #zz2 = list(zip(pitches2,durations2,volumes2,isPauses2))
    #print(len((zz)))
    #M = matrix([[1]])
    Nlen = len(zz)
    
    
    Z = ngrams([t for t in zz[0:Nmaxlen]],Nmaxlen-1)
    Z2 = ngrams([t for t in zz2[0:Nmaxlen]],Nmaxlen-1)
    
    return kernNgram(Z[0],Z2[0])
    
def getVarModel(fn,voice=1,Nmaxlen=100,Ndim=3,Nseq=8,maxlags=6,nCopy=4):
    pitches,durations,volumes,isPauses = parseMidi(fn,part=voice)
    
    #print("consonance: ",measureConsonance(pitches,durations,volumes,isPauses,40))
    #pitches2,durations2,volumes2,isPauses2 = parseMidi(fn2,part=voice)
    
    zz = list(zip(pitches,durations,volumes,isPauses))
    #zz2 = list(zip(pitches2,durations2,volumes2,isPauses2))
    #print(len((zz)))
    #M = matrix([[1]])
    Nlen = len(zz)
    
    ZZ = []

    ZZ.extend(zz[0:Nmaxlen])
    #ZZ.extend(zz[(Nlen//2-Nmaxlen):(Nlen//2+Nmaxlen)])
    #ZZ.extend(zz[-Nmaxlen:])
    
    Z = ngrams([t for t in ZZ],Nseq)
    #Z.extend(ngrams([t for t in zz2[0:Nmaxlen2]],Nseq))
    M = matrix([[kernNgram(t1,t2) for t1 in Z] for t2 in Z],ring=RDF)
    if not M.is_positive_definite():
        M+= matrix.identity(len(Z))*0.1

#print(qq)
#M = matrix([[kern(k1,k2) for k1 in range(88)] for k2 in range(88)])
    #print(M.str())
#print(M.is_positive_definite())

    #print(M)
    CC = M.cholesky()
#Ch = M.cholesky().rows()

    from sklearn.decomposition import PCA

    nDim =Ndim
    pca = PCA(n_components=nDim)
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    stdScaler = StandardScaler()
    Ch = pca.fit_transform(stdScaler.fit_transform(CC))

    model = VAR(Ch)

    results = model.fit(maxlags=maxlags,trend="nc",ic="aic")
    #results = model.fit(maxlags,trend="nc")
    #print(results.summary())

    lag_order = results.k_ar

    preds = []
    X = [x for x in Ch]
    return results, X,Ch,Z,CC




def getCholesky(fn,voice=1,Nmaxlen=100,Ndim=3,Nseq=8,maxlags=6,nCopy=4):
    pitches,durations,volumes,isPauses = parseMidi(fn,part=voice)
    
    #print("consonance: ",measureConsonance(pitches,durations,volumes,isPauses,40))
    #pitches2,durations2,volumes2,isPauses2 = parseMidi(fn2,part=voice)
    
    zz = list(zip(pitches,durations,volumes,isPauses))
    #zz2 = list(zip(pitches2,durations2,volumes2,isPauses2))
    print("len(zz) = ",len((zz)))
    #M = matrix([[1]])
    Nlen = len(zz)
    
    ZZ = []

    ZZ.extend(zz[0:Nmaxlen])
    #ZZ.extend(zz[(Nlen//2-Nmaxlen):(Nlen//2+Nmaxlen)])
    #ZZ.extend(zz[-Nmaxlen:])
    
    Z = ngrams([t for t in ZZ],Nseq)
    #Z.extend(ngrams([t for t in zz2[0:Nmaxlen2]],Nseq))
    M = matrix([[kernNgram(t1,t2) for t1 in Z] for t2 in Z],ring=RDF)
    if not M.is_positive_definite():
        M+= matrix.identity(len(Z))*0.01

#print(qq)
#M = matrix([[kern(k1,k2) for k1 in range(88)] for k2 in range(88)])
    #print(M.str())
#print(M.is_positive_definite())

    #print(M)
    CC = M.cholesky()
#Ch = M.cholesky().rows()

    from sklearn.decomposition import PCA

    nDim =Ndim
    pca = PCA(n_components=nDim)
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    stdScaler = StandardScaler()
    Ch = pca.fit_transform(stdScaler.fit_transform(CC))

    #model = VAR(Ch)

    #results = model.fit(maxlags=maxlags,trend="nc",ic="aic")
    #results = model.fit(maxlags,trend="nc")
    #print(results.summary())

    #lag_order = results.k_ar

    preds = []
    X = [x for x in Ch]
    return  X,Ch,Z,CC


def kk(a,b):
    return gcd(a,b)^2/(a*b)

def getData(Ndim,numbers=range(1,100),kern=kk):
    M = matrix([[kern(a,b) for a in numbers] for b in numbers],ring=RDF)
    if not M.is_positive_definite():
        M += 0.1*matrix.identity(len(numbers))
    CC = M.cholesky()
    
    from sklearn.decomposition import PCA
    nDim =Ndim
    pca = PCA(n_components=nDim)
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    stdScaler = StandardScaler()
    Ch = pca.fit_transform(stdScaler.fit_transform(CC))
    X = [x for x in Ch]
    return X
    
def forecastData(fn,voice=1,Nmaxlen=100,Ndim=3,Nseq=10,maxlags=6,nCopy=4,numbers=range(1,100),kern=kk,useThisDataX=None):
    X1,Ch1,Z1,CC1 = getCholesky(fn,voice[0],Nmaxlen,Ndim,Nseq,maxlags,nCopy)
    #result2,X2,Ch2,Z2,CC2 = getVarModel(fn2,voice[1],Nmaxlen,Ndim,Nseq,maxlags,nCopy)
    if useThisDataX is None:
        X1 = getData(Ndim,numbers,kern)
        Nforecast = len(numbers)
    else:
        X1 = useThisDataX
        Nforecast = len(X1)
    #lag_order = result1.k_ar
    preds = []
    
    
    
    for k in range(Nforecast):
        preds.append(X1[k])

    from sklearn.neighbors import NearestNeighbors

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(Ch1)

    def findBestMatch(nbrs,new_row):
        distances,indices = nbrs.kneighbors([np.array(new_row)])
        return indices[0][0]


    inds = []
    for x in preds:
        i = findBestMatch(nbrs,x)
        
        lz = len(Z1[i])//2
        
        inds.extend(Z1[i][lz-nCopy:lz+nCopy+1])

    return(inds)

    
    
def forecastPart(fn,fn2,voice=1,Nforecast=20,Nmaxlen=100,Ndim=3,Nseq=8,maxlags=6,nCopy=4):
    result1,X1,Ch1,Z1,CC1 = getVarModel(fn,voice[0],Nmaxlen,Ndim,Nseq,maxlags,nCopy)
    result2,X2,Ch2,Z2,CC2 = getVarModel(fn2,voice[1],Nmaxlen,Ndim,Nseq,maxlags,nCopy)
    #X1 = getData(Nforecast,Ndim)
    lag_order = result2.k_ar
    preds = []
    
    
    for k in range(Nforecast):
        a2 = 0.9
        pred = result2.forecast(np.array((X1[k:(k+lag_order)])), 1)
        preds.append(pred[0])
        X1.append(pred[0])

    from sklearn.neighbors import NearestNeighbors

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(Ch1)

    def findBestMatch(nbrs,new_row):
        distances,indices = nbrs.kneighbors([np.array(new_row)])
        return indices[0][0]


    inds = []
    for x in preds:
        i = findBestMatch(nbrs,x)       
        lz = len(Z1[i])//2
        inds.extend(Z1[i][lz-nCopy:lz+nCopy+1])

    return(inds)

durationslist = [[sum([(2**(n-i)) for i in range(d+1)]) for n in range(-8,3+1)] for d in range(1,2)]
notevalues = []
for i in range(len(durationslist)):
    notevalues.extend(durationslist[i])
    
notevalues = sorted(notevalues)    
print(notevalues)
print(len(notevalues))

def findNearestDuration(duration):
    return sorted([(abs(duration-nv),nv) for nv in notevalues])[0][1]

def writePitches(fn,inds,tempo=82,instrument=[0,0]):
    from MidiFile import MIDIFile

    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 1   # In beats
    tempo    = tempo # In BPM
    volume   = 116 # 0-127, as per the MIDI standard

    ni = len(inds)
    MyMIDI = MIDIFile(ni,adjust_origin=False) # One track, defaults to format 1 (tempo track
                     # automatically created)
    MyMIDI.addTempo(track,time, tempo)


    for k in range(ni):
        MyMIDI.addProgramChange(k,k,0,instrument[k])


    times = ni*[0]
    for k in range(len(inds)):
        channel = k
        track = k
        for i in range(len(inds[k])):
            pitch,duration,volume,isPause = inds[k][i]
            track = k
            channel = k
            duration = duration*12*4 #findNearestDuration(duration*12*4)
            #print(k,pitch,times[k],duration,100)
            if not isPause: #rest
                #print(volumes[i])
                # because of median:
                pitch = int(floor(pitch))
                MyMIDI.addNote(track, channel, pitch+21, float(times[k]) , float(duration), int(127*volume))#*(ni-k+10.0)/(ni+10.0))
            times[k] += duration*1.0    
       
    with open(fn, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    print("written")    
 



def getMostSimilar(fn,fns,parts = [0,1]):
    return sorted([(sum([similarity(fn,f,part=part) for part in parts]),f) for f in fns if f!=fn],reverse=True)[0][1]



def midiWithData(fn,pps,instruments=[0]*3,numbers=range(1,100),kern=kk,useThisDataX=None,Ncopys=[0]):
    
    iinds = []
    for i in range(len(pps)):
        p = pps[i]
        print(p)
        inds = forecastData(fn,voice=p,Nmaxlen=200,
                            Ndim=11,Nseq=19,maxlags=6,nCopy=Ncopys[i],
                            numbers=numbers[i],kern=kern,useThisDataX=useThisDataX)
        iinds.append(inds)
    fn = fn+".mix.mid"    
    writePitches(fn,iinds,tempo=80,instrument=instruments)

def rewrite(str,rules):
    s = ""
    for x in str:
        s += rules[x]
    return s

def iterRewrite(str,rules,n=1):
    s = str
    for x in range(n):
        s = rewrite(s,rules)
    return s    

    
def mixMidis(fn1,fn2,instruments=[0]*9,parts=list(zip(list(range(2)),[0,1])), nCopy=[4,0]):
    #pps = [0,1,2,3,4,5]
    pps = parts 
    print(pps)
    iinds = []
    for i in range(len(pps)):
        p = pps[i]
        print(p)
        inds = forecastPart(fn1,fn2,voice=p,Nforecast=200,Nmaxlen=200,Ndim=7,Nseq=7,maxlags=4,nCopy=nCopy[i])
    #print(measureConsonance(pitches,durations,volumes,isPauses,Nmaxlen=23))
    #print(inds)
        iinds.append(inds)
    fn1 = fn1.split("/")[-1]
    fn2 = fn2.split("/")[-1]
    fn = "./midi/"+fn1+"_"+fn2+".mix.mid"    
    writePitches(fn,iinds,tempo=80,instrument=instruments) #[0]*len(iinds))    

#from constants import INSTRUMENT_MAP as im, DRUM_MAP as dm

instruments = []

def kk(a,b):
    return gcd(a,b)**2/(a*b)

def kk2(a,b):
    return 2*gcd(a,b)/(a+b)

def kk3(a,b):
    return 2*gcd(a,b)**3/(a*b*(a+b))


def mainNumbersData():
    pass

def mainImageData(fn):
    imageFN = "./data/two.jpg"
    div = divisors(read_image(imageFN).shape[1]) # divisors of the length in y-direction of the image
    nr = sorted([ (abs(d-100),d) for d in div])[0][1] # the closest divisor to the number 50
    print(nr)
    data = constructImageData(Ndim=19,imageFN=imageFN,nr=nr)
    useThisDataX = data
    pps = list(zip([0,1],[0,0]))
    kk3 = None
    midiWithData(fn,pps,instruments=2*[0],numbers=numbers,kern=kk3,useThisDataX=useThisDataX,Ncopys=[3,1])


def mainTSPData(fn):
    data = constructData(Ndim=11) 
    permutation,distance = tsp(data)
    data = [data[permutation[k]] for k in range(len(data))]
    useThisDataX = data
    pps = list(zip([0,1],[0,0]))
    kk3 = None
    midiWithData(fn,pps,instruments=2*[0],numbers=numbers,kern=kk3,useThisDataX=useThisDataX,Ncopys=[3,1])


def mainMixMidis(fn1,fn2):
    mixMidis(fn1,fn2,instruments=2*[0],parts=list(zip([0,1],[0,1])),nCopy=[3,2])


def main():
    import os
    fn1 = sorted([os.path.join("./midi/",fn) for fn in os.listdir("./midi/") if fn.endswith(".mid") 
              and "clair_de_lune_theme" in fn.lower() and not "mix" in fn.lower()])[0]
    
    fn2 = sorted([os.path.join("./midi/",fn) for fn in os.listdir("./midi/") if fn.endswith(".mid") 
              and "swan_theme" in fn.lower() and not "mix" in fn.lower() and not "mix" in fn.lower()])[0]
    
    #mainTSPData(fn1)
    #mainImageData(fn1)
    mainMixMidis(fn1,fn2)
    

main()    





[3/512, 3/256, 3/128, 3/64, 3/32, 3/16, 3/8, 3/4, 3/2, 3, 6, 12]
12
[(0, 0), (1, 1)]
(0, 0)
(1, 1)
written


In [19]:
print(list(Permutations([0,1,2])))

[[0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 0, 1], [2, 1, 0]]


In [17]:
print([int(floor(13*sin(x*pi.n()/(2*7)).n()))+14 for x in range(1,401)])

[16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 14, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 14, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 14, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 14, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 14, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 13, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 14, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 13, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 14, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 8, 11, 13, 16, 19, 22, 24, 25, 26, 27, 26, 25, 24, 22, 19, 16, 13, 11, 8, 5, 3, 2, 1, 1, 1, 2, 3, 5, 